In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath('__file__')))

from IPython.display import display, Markdown
import ipywidgets as widgets

from knowledge_base import KNOWLEDGE_RULES, SYSTEM_KNOWLEDGE_TEXT, SYSTEM_PROMPT
from analyzer import (
    SystemPromptAnalyzer,
    list_models,
    OLLAMA_BASE_URL,
)

print('✅ 模块加载成功')
print(f'   知识库规则数: {len(KNOWLEDGE_RULES)}')
print(f'   Ollama 地址 : {OLLAMA_BASE_URL}')

✅ 模块加载成功
   知识库规则数: 8
   Ollama 地址 : http://localhost:11434


In [3]:
# ── 检查 Ollama 连接状态 ──────────────────────────
try:
    models = list_models()
    display(Markdown('### ✅ Ollama 连接正常'))
    display(Markdown('**本地可用模型：**'))
    for m in models:
        print(f'  • {m}')
except Exception as e:
    display(Markdown(f'### ❌ 无法连接 Ollama\n> 请先执行 `ollama serve`\n\n错误：`{e}`'))

### ✅ Ollama 连接正常

**本地可用模型：**

  • qwen3:4b-instruct
  • qwen3:4b


In [4]:
# ── 全局配置（按需修改）──────────────────────────
CONFIG = {
    'model':       'qwen3:4b',     # 对话模型（改成你本地已 pull 的模型）
    'temperature': 0.1,                # 低温度 = 更确定性的输出
}
print('当前配置：')
for k, v in CONFIG.items():
    print(f'  {k:15s} = {v}')

当前配置：
  model           = qwen3:4b
  temperature     = 0.1


In [5]:

# 内置示例日志（覆盖所有根因类型）
DEMO_LOGS = {
    'Mac JBR Metal崩溃': """
java.lang.IllegalStateException: Error - unable to initialize Metal after
recreation of graphics device. Cannot load metal library: No MTLDevice.
java.desktop/sun.awt.CGraphicsDevice.<init>(CGraphicsDevice.java:91)
Exception in NSApplicationAWT: java.lang.IllegalStateException: ...
""",
    'Windows虚拟内存不足': """
Native memory allocation (malloc) failed to allocate 1407664 bytes.
Error detail: Chunk::new
Out of Memory Error (arena.cpp:191), pid=2680, tid=9240
""",
    '物理内存不足': """
# Native memory allocation (malloc) failed to allocate 1330048 bytes.
# Error detail: Chunk::new
# Possible reasons:
#   The system is out of physical RAM or swap space
#   This process is running with CompressedOops enabled,
#   and the Java Heap may be blocking the growth of the native heap
""",
    'chrome_elf访问违例': """
EXCEPTION_ACCESS_VIOLATION (0xc0000005) at pc=0x0000000000000000, pid=928, tid=5776
# Problematic frame:
# C  [chrome_elf.dll+0x1b549]  java.lang.ProcessHandleImpl.getProcessPids0
""",
    'GC线程崩溃（疑似CPU）': """
EXCEPTION_ACCESS_VIOLATION (0xc0000005) at pc=0x00007ffd4c6c2580, pid=33548, tid=4488
# Problematic frame:
# V  [jvm.dll+0x3f6d67]
Current thread (0x000002617bfc3730): GCTaskThread "GC Thread#5"
[stack: 0x000000777e600000,0x000000777e700000] [id=22192]
""",
    'JBR-A-27偶发崩溃': """
# EXCEPTION_ACCESS_VIOLATION (0xc0000005) at pc=0x00007ffcaed3c475, pid=17708, tid=5556
# JRE version: OpenJDK Runtime Environment JBR-17.0.12+1-1087.25-jcef (17.0.12+1)
# Java VM: OpenJDK 64-Bit Server VM JBR-17.0.12+1-1087.25-jcef
# Problematic frame:
# V  [jvm.dll+0x36c475]
""",
    'JBR空指针（backBuffers）': """
java.lang.NullPointerException: Cannot invoke
"java.awt.image.VolatileImage.getGraphics()"
because "this.backBuffers[i]" is null
""",
    'JDK Bug（字体渲染）': """
EXCEPTION_ACCESS_VIOLATION (0xc0000005) at pc=0x00007ffacb926d67, pid=21720, tid=12544
# Problematic frame:
# V  [jvm.dll+0x3f6d67]  sun.java2d.loops.DrawGlyphListLCD.DrawGlyphListLCD()
""",
}

print(f'共 {len(DEMO_LOGS)} 条示例日志：')
for i, name in enumerate(DEMO_LOGS, 1):
    print(f'  {i}. {name}')

共 8 条示例日志：
  1. Mac JBR Metal崩溃
  2. Windows虚拟内存不足
  3. 物理内存不足
  4. chrome_elf访问违例
  5. GC线程崩溃（疑似CPU）
  6. JBR-A-27偶发崩溃
  7. JBR空指针（backBuffers）
  8. JDK Bug（字体渲染）


In [6]:

# 初始化分析器
sp_analyzer = SystemPromptAnalyzer(model=CONFIG['model'])

# 查看注入的 System Prompt 内容
display(Markdown(f'**System Prompt 长度：** {len(SYSTEM_PROMPT)} 字符 (~{len(SYSTEM_PROMPT)//4} tokens)'))
display(Markdown('**注入内容预览（前 400 字符）：**'))
print(SYSTEM_PROMPT[:400])

[SystemPromptAnalyzer] 使用模型: qwen3:4b
[SystemPromptAnalyzer] 知识注入长度: 4040 字符


**System Prompt 长度：** 4040 字符 (~1010 tokens)

**注入内容预览（前 400 字符）：**


You are an Intelij based IDE crash/error log parser. Given any IDE fatal error log, identify its log type, then extract a crash fingerprint as a JSON object.

## Step 1 — Identify the log type

| Type | How to recognize |
|---|---|
| `hs_err` | Starts with `# A fatal error has been detected by the Java Runtime Environment` |
| `oom` | Contains `There is insufficient memory for the Java Runtime En


In [7]:

# ── 单条日志分析 ─────────────────────────────────
# 从下拉框选择示例，或直接修改 log_to_analyze
# log_to_analyze = DEMO_LOGS['物理内存不足']

with open('java_error_in_devecostudio64_36492.log', 'r', encoding='gbk') as f:
    log_to_analyze = f.readlines()
    log_to_analyze = ''.join(log_to_analyze[:100])  # 将行列表合并成单个字符串

print('📝 待分析日志：')
print(log_to_analyze.strip())
print('\n' + '='*60)
print('🤖 LLM 分析结果（流式输出）：')
print('='*60 + '\n')


📝 待分析日志：
#
# There is insufficient memory for the Java Runtime Environment to continue.
# Native memory allocation (malloc) failed to allocate 32744 bytes. Error detail: ChunkPool::allocate
# Possible reasons:
#   The system is out of physical RAM or swap space
#   This process is running with CompressedOops enabled, and the Java Heap may be blocking the growth of the native heap
# Possible solutions:
#   Reduce memory load on the system
#   Increase physical memory or swap space
#   Check if swap backing store is full
#   Decrease Java heap size (-Xmx/-Xms)
#   Decrease number of Java threads
#   Decrease Java thread stack sizes (-Xss)
#   Set larger code cache with -XX:ReservedCodeCacheSize=
#   JVM is running with Unscaled Compressed Oops mode in which the Java heap is
#     placed in the first 4GB address space. The Java Heap base address is the
#     maximum limit for the native heap growth. Please use -XX:HeapBaseMinAddress
#     to set the Java Heap base and to place the Java He

In [11]:
result = sp_analyzer.analyze(log_to_analyze, stream=True)


[分析中] 模型: qwen3:4b
[分析中] JSON模式: True

{
  "log_type": "oom",
  "exception_type": "OutOfMemoryError",
  "frame_type": "null",
  "frame_fingerprint": "arena.cpp",
  "current_thread": "C2 CompilerThread0",
  "top_frame_method": "null",
  "evidence": [
    "# There is insufficient memory for the Java Runtime Environment to continue.",
    "#  Out of Memory Error (arena.cpp:79), pid=36492, tid=30040"
  ]
}


In [9]:
result

'{\n  "log_type": "oom",\n  "exception_type": "OutOfMemoryError",\n  "frame_type": "null",\n  "frame_fingerprint": "arena.cpp",\n  "current_thread": "C2 CompilerThread0",\n  "top_frame_method": "null",\n  "evidence": [\n    "# There is insufficient memory for the Java Runtime Environment to continue.",\n    "#  Out of Memory Error (arena.cpp:79), pid=36492, tid=30040"\n  ]\n}'